In [1]:
import cv2 
import numpy as np
import sklearn as sk
from matplotlib import pyplot

Implement a trackbar that modifies an intensity threshold value, which will be used in the homogeneity criterion to determine membership in the filled region.
Use of global variables will be necessary for the mouse and trackbar callback functions.

In [2]:
def set_threshold(*args):
    threshold = cv2.getTrackbarPos("Intensity", "Controls")

Calculate the difference between current seed and inital seed
The intensity data type is uint8

In [3]:
def different(x, y):
        if(x<y):
            return y-x
        else:
            return x-y

Check whether the different of the points less/larger than threshold

In [4]:
def change(seed_copy, pix, threshold):
        current = img[pix[0]][pix[1]]
        dif0 = different(seed_copy[0], current[0])
        dif0 = different(seed_copy[1], current[1])
        dif2 = different(seed_copy[2], current[2])
        chag = (dif0 <= threshold) and (dif0 <= threshold) and (dif2 <= threshold)
        return chag

Flood fill function 
I used 4-connectivety to find neighbours
The definition of your homogeneity criterion: Select seeds only from objects of interest (e.g. bright structures, dark structure, color range, etc).

In [5]:
def flood_fill(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:

        threshold = cv2.getTrackbarPos("Intensity", "Controls")
        seed = [y,x]
        seed_copy = np.copy(img[y][x])
        #have to make sure that you don’t visit a pixel more than once, which can lead to an infinite loop.
        check = np.zeros([img_size[0],img_size[1]])
        check[y,x] = 1
        #Create a stack to keep track of possible candidates to fill.
        stack.append([y,x])
        
        #Pop a pixel from your stack, which is the current pixel
        while(len(stack)!=0):
            current = stack.pop()
            change(seed_copy,current, threshold)
            
            img[current[0],current[1]] = [255,0,0]
                
            #find neighbour  
            left_pixel = [max(current[0]-1, 0),current[1]]
            right_pixel = [min(current[0]+1,img_size[0] -1),current[1]] 
            left = [max(current[0]-1, 0)]
            right = [min(current[0]+1,511)]
            up_pixel = [current[0],min(current[1]+1,img_size[1] -1)]              
            down_pixel = [current[0],max(current[1]-1,0)] 
            up = [min(current[1]+1,511)]
            down = [max(current[1]-1,0)]

                
            if(change(seed_copy, down_pixel, threshold) and not check[current[0]][max(current[1]-1,0)]):
                stack.append(down_pixel)
                check[current[0]][max(current[1]-1,0)] = 1
            
            if(change(seed_copy, left_pixel, threshold) and not check[max(current[0]-1, 0)][current[1]]):
                stack.append(left_pixel)
                check[max(current[0]-1, 0)][current[1]] = 1
            
               
            if(change(seed_copy, right_pixel, threshold) and not check[min(current[0]+1,511)][current[1]]):
                stack.append(right_pixel)
                check[min(current[0]+1,511)][current[1]] = 1
            
            if(change(seed_copy, up_pixel, threshold) and not check[current[0]][min(current[1]+1,511)]):
                stack.append(up_pixel)
                check[current[0]][min(current[1]+1,511)] = 1
        


Main function
Import picture and window information
Add two imread function, one is for baboon1.png, the others is my own picture

In [6]:
#img = cv2.imread("baboon1.png")
img = cv2.imread("loulou.png")
#Global Variables
stack = []
threshold = 0
img_size = [img.shape[0], img.shape[1]]
cv2.namedWindow('Controls', cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow('Controls', 800, 50)
cv2.createTrackbar("Intensity", 'Controls', 1, 255, set_threshold)
while True:
    cv2.imshow("Baboon", img)
    cv2.setMouseCallback("Baboon", flood_fill, [stack,img])
    k=cv2.waitKey(1)
    if k==27: 
        break
cv2.destroyAllWindows()